In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import load
import pathlib
import pytz

2023-12-13 19:23:33.937188: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
starting_time_step = datetime(year=2020, month=1, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 900, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
dformer = DataTransformer(env_config=env_config)
dformer.retreive_stocks_buffers()
dformer.retreive_commodities_buffers()

In [2]:
(current_file_path, new_file_path) = DataTransformer.get_new_file_path()

rf = load(pathlib.Path(current_file_path))

nwb_starting_time_step = datetime(year=2023, month=12, day=1)
now_time = pytz.utc.localize(datetime.now())

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 100, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
# display(nwb_dfs[2])
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
nwb_forest = Forester(market=nwb_market)
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step, now_time)
for indx in range(100):
    if indx < 7:
        acts = np.full(100, 15)
        rand_acts = nwb_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        flat_state = np.array([nwb_forest.get_flattened_states(nwb_market.state)])
        my_pred = rf.predict(flat_state)
        acts = np.array([15,15,15,15,15])
        acts = my_pred[0]
        print(nwb_market.the_current_time_step)
        print(acts)
        rand_acts = nwb_forest.act_random(acts)
        print(nwb_market.the_current_time_step.day, now_time.day)
        if nwb_market.the_current_time_step.day >= now_time.day:
            break
        nwb_market.step(rand_acts)

print(nwb_market.wallet_state)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:279: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:280: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:309: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

2100000.0
2023-12-01 00:00:00+00:00 2023-12-13 19:23:40.077664+00:00
2023-12-12 00:00:00+00:00
[ 9. 11. 10. 11.  9. 10. 10. 11. 10. 11. 10. 11. 11. 10. 10. 10. 11. 11.
 10. 11. 10. 11. 11. 11. 10. 11.  9. 10. 10. 11. 10. 10. 11. 10. 11. 10.
 10. 11. 11. 10. 11. 11. 11. 10. 10. 11. 10. 11. 10. 11. 10. 10. 10. 11.
 10.  9. 10. 11. 11. 10. 11. 11. 11. 10. 10. 10. 10. 11.  9. 11.  9. 11.
 10. 10. 11. 10. 11. 10. 11. 11. 10. 10. 11.  8. 10. 10. 11. 10. 10. 10.
  7. 10. 11. 11. 10. 11. 10. 10. 11. 11.]
12 13
2023-12-13 00:00:00+00:00
[ 9. 11. 11. 11.  9. 10. 10. 11. 10. 11. 10.  9. 11. 10. 10. 10. 11. 11.
 10. 11. 10.  9. 11. 11. 10. 11. 11. 10. 11. 11. 10. 10. 11. 10. 11. 10.
 10. 11. 11. 11. 11. 10. 11. 10. 10. 11. 10. 11. 10. 11. 10. 10.  8. 11.
  9.  9. 10. 11. 11. 10. 11. 11. 11. 10. 10. 10. 10. 11.  9. 11. 12. 11.
 10.  8. 11. 10. 11. 10. 11. 11. 11.  9. 11.  8. 10. 10. 11. 10. 11.  7.
 10. 10. 11. 11. 11. 11. 11. 10. 11. 11.]
13 13
0.0
